In [2]:
import pandas as pd
import numpy as np

In [3]:
# defining betting related functions
def seriesOdds(team1Blue, team2Blue, best_of):
    if best_of == 1:
        return max(team1Blue, team2Blue)
    if best_of == 3:
        a = max(team1Blue, 1-team2Blue)
        b = max(1-team1Blue, team2Blue)
        return a * (1-b) + a * a * b + a * (1-a) * (1-b)
    if best_of == 5:
        a = max(team1Blue, 1-team2Blue)
        b = max(1-team1Blue, team2Blue)
        p1 = a * (1-b) * (1-b)
        p2 = 2*a*a*b*(1-b) + a*(1-a)*(1-b)*(1-b)
        p3 = a*a*a*b*b + 4*a*a*b*(1-a)*(1-b) + a*(1-a)*(1-a)*(1-b)*(1-b)
        return p1 + p2 + p3

# -1 for red, 1 for blue, 0 for neither
def betByOdds(predictions, blue_odds, red_odds, threshold = 0):
    blue_threshold = 1 / blue_odds
    red_threshold = 1 / red_odds

    if predictions > 0.5:
        if predictions > blue_threshold + threshold:
            return 1
        return 0

def oddsConverter(odds):
    if odds < 0:
        return (100 - odds) / -odds
    else:
        return (100 + odds) / 100

In [4]:
predictions = pd.read_csv("predictions/lck_summer_2024.csv")
predictions


,Blue,Red,Prediction
0,T1,KT Rolster,0.664815
1,Hanwha Life eSports,Nongshim RedForce,0.692052
2,OK BRION,FearX,0.344523
3,Kwangdong Freecs,Dplus KIA,0.479264


In [5]:
odds = pd.DataFrame(data = [[1.14,5.2],
                            [1.08,6.8],
                            [2.41,1.53],
                            [3,1.36]], columns = ["Blue_Odds", "Red_Odds"])

In [6]:
predictionOdds = pd.concat([predictions, odds], axis = 1)


In [7]:
threshold = 0.15
minodds = 1.03

predictionOdds["To_Bet"] = predictionOdds.apply(lambda row: abs(row.Prediction - 0.5) > threshold and min(row.Blue_Odds, row.Red_Odds) > minodds, axis = 1)
predictionOdds


,Blue,Red,Prediction,Blue_Odds,Red_Odds,To_Bet
0,T1,KT Rolster,0.664815,1.14,5.20,True
1,Hanwha Life eSports,Nongshim RedForce,0.692052,1.08,6.80,True
2,OK BRION,FearX,0.344523,2.41,1.53,True
3,Kwangdong Freecs,Dplus KIA,0.479264,3.00,1.36,False


In [8]:
predictionOdds.to_csv("predictions/lck_summer_bets.csv")